In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import xmltodict
import pandas as pd
import time
import os

In [ ]:
place_50_gangnam = ["강남 MICE 관광특구","압구정로데오거리","가로수길","역삼역","강남역","선릉역"]
place_50_guro = ["신도림역","구로디지털단지역"]
place_50_jongro = ["창덕궁·종묘","광화문·덕수궁","경복궁·서촌마을","낙산공원·이화마을","북촌한옥마을","종로·청계 관광특구","인사동·익선동"]
place_50_mapo = ["망원한강공원","월드컵공원","신촌·이대역","홍대 관광특구"]
place_50_yongsan = ["용산역","국립중앙박물관·용산가족공원","남산공원","이태원 관광특구","서울역","이촌한강공원"]

In [ ]:
# [6, 2, 7, 4, 6]
[len(place_50_gangnam), len(place_50_guro),len(place_50_jongro),len(place_50_mapo),len(place_50_yongsan)]

In [ ]:
KEY = ""
TYPE = "XML"
PLACES = ['강남 MICE 관광특구', '동대문 관광특구', '명동 관광특구', '이태원 관광특구', '잠실 관광특구',
       '종로·청계 관광특구', '홍대 관광특구', '경복궁·서촌마을', '광화문·덕수궁', '창덕궁·종묘',
       '가산디지털단지역', '강남역', '건대입구역', '고속터미널역', '교대역', '구로디지털단지역', '서울역',
       '선릉역', '신도림역', '신림역', '신촌·이대역', '역삼역', '연신내역', '용산역', '왕십리역',
       'DMC(디지털미디어시티)', '창동 신경제 중심지', '노량진', '낙산공원·이화마을', '북촌한옥마을',
       '가로수길', '성수카페거리', '수유리 먹자골목', '쌍문동 맛집거리', '압구정로데오거리', '여의도',
       '영등포 타임스퀘어', '인사동·익선동', '국립중앙박물관·용산가족공원', '남산공원', '뚝섬한강공원',
       '망원한강공원', '반포한강공원', '북서울꿈의숲', '서울대공원', '서울숲공원', '월드컵공원', '이촌한강공원',
       '잠실종합운동장', '잠실한강공원']
index_list = ['WEATHER_TIME', 'TEMP', 'SENSIBLE_TEMP', 'MAX_TEMP', 'MIN_TEMP', 'HUMIDITY', 'WIND_DIRCT', 'WIND_SPD', 
              'PRECIPITATION', 'PRECPT_TYPE', 'PCP_MSG', 'SUNRISE', 'SUNSET', 'UV_INDEX_LVL', 'UV_INDEX', 
              'UV_MSG', 'PM25_INDEX', 'PM25', 'PM10_INDEX', 'PM10', 'AIR_IDX', 'AIR_IDX_MVL', 'AIR_IDX_MAIN', 'AIR_MSG']
CSV_FILE_NAME = "seoul_50.csv"

In [ ]:
def getSoup(url):
    import requests
    from bs4 import BeautifulSoup
    
    soup = ""
    try:
        response = requests.get(url)
        time.sleep(1)
        if response.status_code != 200:
            raise Exception("HTTP 코드 이상")
        soup = BeautifulSoup(response.content, "lxml-xml")
    except:
        print("Error")
        soup = getSoup(url)
    return soup

In [ ]:
def getData(place, KEY, TYPE):
    url = f"http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/citydata/1/5/{place}"
    soup = getSoup(url)

    # 1.장소이름 추가
    data = {}            
    tmp = soup.find("AREA_NM")
    while isinstance(tmp, type(None)):
        soup = getSoup(url)
        tmp = soup.find("AREA_NM")      
        print(f'에러1 발생: {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}')
    data["AREA_NM"] = soup.find("AREA_NM").get_text()

    # 2
    tmp = soup.find("CITYDATA").find("LIVE_PPLTN_STTS").find("LIVE_PPLTN_STTS")
    while isinstance(tmp, type(None)):
        soup = getSoup(url)
        tmp = soup.find("CITYDATA").find("LIVE_PPLTN_STTS").find("LIVE_PPLTN_STTS")
        print(f'에러2 발생: {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}')
    tmp = xmltodict.parse(str(tmp))["LIVE_PPLTN_STTS"]
    data = dict(data, **tmp)

    # 3
    result = {}
    tmp = soup.find("WEATHER_STTS").find("WEATHER_STTS")
    while isinstance(tmp, type(None)):
        soup = getSoup(url)
        tmp = soup.find("WEATHER_STTS").find("WEATHER_STTS")
        print(f'에러3 발생: {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}')
    tmp = xmltodict.parse(str(tmp))["WEATHER_STTS"]
    for index in index_list: # 원하는 칼럼만 뽑아내기
        a = tmp[index]
        result[index] = a
    data = dict(data, **result)   

    # 4 현재 시간 추가
    data["NOW"] = time.strftime("%Y-%m-%d %H:%M", now)
    return data

In [ ]:
def save(data, CSV_FILE_NAME):
    import pandas as pd
    import os
    # 데이터프레임 변환 후 csv파일에 저장
    df = pd.DataFrame(data = [list(data.values())], columns = list(data.keys()))    
    if not os.path.exists(CSV_FILE_NAME):
        df.to_csv(CSV_FILE_NAME, index=False)
    else:
        df.to_csv(CSV_FILE_NAME, index=False, mode = 'a', header=False)      

# 실행

In [ ]:
# 실행
while True:
    now = time.localtime()    
    if now.tm_min in [2,12,22,32,42,52]:  
        for place in PLACES:
            data = getData(place, KEY, TYPE)
            save(data, CSV_FILE_NAME)               
    else:
        time.sleep(10)